In [ ]:
import os
import pandas as pd

folder_path = "./dataset/labels/val"

file_names = []
file_contents = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        with open(os.path.join(folder_path, file_name), "r") as file:
            content = file.read()
        file_names.append(file_name)
        file_contents.append(content)

df = pd.DataFrame({"filename": file_names, "file_contents": file_contents})

print(df)


In [1]:
import numpy as np
def iou(box1, box2):
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [2]:
def translate_bbox(bbox):
    img_size = 1024 # rsna defualt image size
    
    top_left_x = bbox[0]
    top_left_y = bbox[1]
    absolute_w = bbox[2]
    absolute_h = bbox[3]

    relative_w = absolute_w / img_size
    relative_h = absolute_h / img_size
    
    relative_x = top_left_x / img_size + relative_w / 2
    relative_y = top_left_y / img_size + relative_h / 2
    
    return relative_x, relative_y, relative_w, relative_h
    
def revert_bbox(rx, ry, rw, rh):
    img_size = 1024 # rsna defualt image size
    
    x = (rx-rw/2)*img_size
    y = (ry-rh/2)*img_size
    w = rw*img_size
    h = rh*img_size
    
    return x, y, w, h

In [ ]:
def evaluate(filename, xywh):
    target_str = df[df['filename'] == filename]
    if target_str == '':
        return 0
    elements = target_str.split()
    sublists = [elements[i:i+5] for i in range(0, len(elements), 5)]
    bbox_targets = [tuple(map(float, sublist[-4:])) for sublist in sublists]
    result = []
    for bbox_target in bbox_targets:
        bbox1 = translate_bbox(bbox_target)
        bbox2 = translate_bbox(xywh)
        result.append(iou(bbox1, bbox2))

    return max(result)
    